# Import Necessary Libraries and Files

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier

import torch
import torchvision
from torchvision import transforms, utils
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from collections import Counter
from PIL import Image
from skimage import io, transform
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from collections import Counter

!pip install kaggle

from google.colab import drive




import gensim.downloader as api





In [ ]:
#Set up GPU capabilities
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using '{device}' device")

Using 'cuda' device


In [ ]:
#Import Dataset

from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

# Specify the correct path to file on Google Drive - MIGHT REQUIRE MODIFICATION FOR 3RD PARTIES!
file_url = '/content/drive/MyDrive/IMDB Dataset.csv'

# Output path in Colab
output_path = '/content/IMDB Dataset.csv'

# Copy the file from Google Drive to Colab
!cp "$file_url" "$output_path"

# Load the dataset using Pandas
df = pd.read_csv(output_path)

# Display the first few rows of the dataset
print(df.head())

print(df.info())

Mounted at /content/drive
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None


# Set up Datasets

In [ ]:
#'review' is the feature column and 'sentiment' is the label column
X = df['review']
y = df['sentiment']

# Convert string labels to binary format
y_binary = y.apply(lambda label: 1 if label == 'positive' else 0)

# Create a new DataFrame with the processed labels
df_processed = pd.DataFrame({'review': X, 'sentiment': y_binary})

# Create a subset for prototyping
subset_size = 20000
#df_processed = df_processed.head(subset_size)
df_processed = df_processed


# Split the subset into training, validation, and testing sets
train_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
train_df, validation_df = train_test_split(train_df, test_size=0.2, random_state=42)



#Prepare Data for Dataloader

In [ ]:
#Install torchtext
!pip install torchtext==0.6

from torchtext import data
from torchtext.data import Field, TabularDataset, BucketIterator
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.1 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


In [ ]:

# Set the seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Download the NLTK punkt tokenizer model
nltk.download('punkt')

# Tokenization and Field definition with NLTK

TEXT = Field(tokenize=word_tokenize, include_lengths=True)
LABEL = Field(sequential=False, use_vocab=False, is_target=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from torch.utils.data import Subset

# Specify the path to CSV file
csv_path = '/content/IMDB Dataset.csv'

# Convert DataFrames to CSV files
train_df.to_csv('/content/train.csv', index=False)
validation_df.to_csv('/content/validation.csv', index=False)
test_df.to_csv('/content/test.csv', index=False)

# Create TabularDataset
train_dataset, validation_dataset, test_dataset = TabularDataset.splits(
    path='/content/',
    train='train.csv',
    validation = 'validation.csv',
    test='test.csv',
    format='csv',
    fields=[('review', TEXT), ('sentiment', LABEL)],
    skip_header=True
)




In [ ]:
print(len(train_dataset))
print(len(validation_dataset))
print(len(test_dataset))

32000
8000
10000


In [ ]:
# Inspect the first training example
print(vars(train_dataset.examples[0]))
print(vars(train_dataset.examples[len(train_dataset)-1]))

# Inspect the first validation example
print(vars(validation_dataset.examples[0]))

# Inspect the first testing example
print(vars(test_dataset.examples[0]))

# Accessing the fields
print(train_dataset.fields)

{'review': ['With', 'no', 'fault', 'to', 'the', 'actors', '(', 'they', 'all', 'put', 'on', 'great', 'performances', ')', ',', 'the', 'overall', 'story', 'was', 'not', 'very', 'well', 'executed', '.', 'The', 'movie', 'opens', 'with', 'a', 'great', 'zinger', ':', 'a', 'crazy', 'old', 'guy', 'forces', 'a', 'young', 'Aborigine', 'girl', "'s", 'car', 'off', 'the', 'road', '.', 'But', 'then', ',', 'we', "'re", 'forced', 'to', 'endure', '40', 'minutes', 'of', 'character', 'development', 'with', 'an', 'entirely', 'new', 'group', 'of', 'characters', '...', 'and', 'we', 'do', "n't", 'know', 'why', 'until', 'the', '40', 'minutes', 'are', 'up', '.', 'It', 'turns', 'out', 'that', 'they', 'are', 'the', 'ones', 'who', 'eventually', 'discover', 'the', 'girl', "'s", 'body', '...', 'and', 'the', 'story', 'progresses', 'from', 'there.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'While', 'the', 'story', 'does', 'pick', 'up', 'at', 'that', 'point', ',', 'it', 'really', 'goes', 'nowhere', '.', 'After', '2',

In [ ]:
# Build vocabulary
TEXT.build_vocab(train_dataset, max_size=25000, vectors="glove.6B.100d", unk_init=torch.Tensor.normal_)
input_dim = len(TEXT.vocab)
print(input_dim)

LABEL.build_vocab(train_dataset)

25002


#Data Loader

In [ ]:
batch = 128

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence


train_iterator, validation_iterator, test_iterator = data.BucketIterator.splits(
      (train_dataset, validation_dataset, test_dataset),
      batch_size=batch,
      sort_within_batch=True,
      sort_key=lambda x: len(x.review),
      device=device
  )


#Create RNN Class

In [ ]:
# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        # Embedding layer to convert input tokens to dense vectors of fixed size
        self.embedding = nn.Embedding(input_dim, embedding_dim)

        # Bidirectional LSTM layer to capture sequential patterns in both directions
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, bidirectional=True)

        # Fully connected layer to produce the final output
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # *2 because of bidirectional LSTM

        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.7)

    def forward(self, text, text_lengths):
        # Embedding layer converts input tokens to dense vectors and applies dropout
        embedded = self.dropout(self.embedding(text))

        # Pack the padded sequence to optimize processing in the recurrent layer
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)

        # Bidirectional LSTM processes the packed sequence
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        # Unpack the sequence to obtain the output
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        # Concatenate the final hidden states from both directions
        hidden = self.dropout(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))

        # Pass the concatenated hidden states through the fully connected layer
        return self.fc(hidden)

# Train RNN

In [ ]:
def train_and_validate(model: RNN, train_iterator: DataLoader, validation_iterator: DataLoader, epochs: int, optimizer: optim.Optimizer, loss_fn: nn.Module, device: torch.device) -> None:
    """
    Trains the model and validates it on a validation set for the specified number of epochs
    Inputs
    ------
    ------
    model: RNN model to train
    train_iterator: DataLoader for training data
    validation_iterator: DataLoader for validation data
    epochs: Number of epochs to train the model
    optimizer: Optimizer to use for each epoch
    loss_fn: Function to calculate loss
    device: Device to move tensors to (e.g., 'cuda' or 'cpu')
    """
    train_losses = {}
    validation_losses = {}

    model.to(device)
    model.train()

    print("=> Starting training")

    for epoch in range(epochs):
        epoch_losses = []

        for batch in train_iterator:
            # Move input data and labels to the specified device
            text, text_lengths = batch.review
            labels = batch.sentiment
            text, labels = text.to(device), labels.to(device)
            # Move lengths to CPU
            text_lengths = text_lengths.cpu()

            # Clear gradients
            optimizer.zero_grad()

            # Initialize hidden states to zeros
            hidden = torch.zeros(2, len(text), model.rnn.hidden_size).to(device)  # Assuming 2 for bidirectional LSTM

            # Forward pass
            outputs = model(text, text_lengths)

            # Calculate the loss
            loss = loss_fn(outputs.squeeze(1), labels.float())

            # Backward pass and optimization
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 3)  # Clip gradients to avoid exploding gradients
            optimizer.step()

            epoch_losses.append(loss.item())

        train_losses[epoch] = torch.tensor(epoch_losses).mean()
        print(f'=> Epoch: {epoch + 1}, Loss: {train_losses[epoch]}')

        # Validation
        model.eval()
        validation_loss = evaluate(model, validation_iterator, loss_fn, device)
        validation_losses[epoch] = validation_loss
        #print(f'=> Epoch: {epoch + 1}, Validation Loss: {validation_loss}')


        # Set the model back to training mode
        model.train()

    return validation_loss

In [ ]:
def evaluate(model, test_iterator, loss_fn, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for batch in test_iterator:
            text, text_lengths = batch.review
            labels = batch.sentiment
            text, labels = text.to(device), labels.to(device)

            # Move lengths to CPU
            text_lengths = text_lengths.cpu()

            outputs = model(text, text_lengths)
            loss = loss_fn(outputs.squeeze(1), labels.float())

            total_loss += loss.item()

            # Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
            predictions = (torch.sigmoid(outputs) >= 0.5).float()

            # Count correct predictions for each sample in the batch
            for i in range(len(predictions)):
                correct_predictions += (predictions[i] == labels[i]).item()


            total_samples += labels.size(0)

            # Print a few predictions for inspection
            #print("Predictions:", predictions.cpu().numpy())
            #print("True Labels:", labels.cpu().numpy())
            #print("Correct Predictions",correct_predictions)

    average_loss = total_loss / len(test_iterator)
    accuracy = correct_predictions / total_samples

    print(f'=> Validation/Test Accuracy: {accuracy * 100:.2f}%')
    #print(f'=> Evaluation Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')
    #print(total_loss)
    #print(accuracy)
    #print(correct_predictions)
    #print(total_samples)
    #print(len(test_iterator))

    return average_loss




# Use Validation Set to Tune Hyperparameters

In [ ]:
import itertools

# Initialize variables to keep track of the best hyperparameters and corresponding validation loss
best_hyperparameters = None
best_validation_loss = float('inf')
validation_loss = 0

# Define hyperparameter options
embedding_dims = [100, 200, 300]
hidden_dims = [128, 256, 512]
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]
weight_decays = [1e-5, 1e-4, 1e-3]
output_dim = 1

#Define Loss Function
loss_fn = nn.BCEWithLogitsLoss() #Binary Cross Entropy with logits

# Create a list of all combinations
hyperparameter_combinations = list(itertools.product(embedding_dims, hidden_dims, learning_rates, batch_sizes, weight_decays))

# Randomly sample a subset of combinations
num_random_combinations = 10
random_combinations = random.sample(hyperparameter_combinations, num_random_combinations)

# Iterate through the randomly selected combinations
for combination in random_combinations:
    emb_dim, hid_dim, lr, batch_size, weight_decay = combination


    #Define model with selected hyperparameters
    rnn_model = RNN(input_dim=input_dim, embedding_dim=emb_dim, hidden_dim=hid_dim, output_dim=output_dim)

    # Create a new optimizer with the current learning rate
    optimizer = optim.Adam(rnn_model.parameters(), lr = lr, weight_decay=weight_decay)

    # Create a new DataLoader with the current batch size for both training and validation
    train_iterator, validation_iterator = data.BucketIterator.splits(
        (train_dataset, validation_dataset),
        batch_size=batch_size,
        sort_within_batch=True,
        sort_key=lambda x: len(x.review),
        device=device
    )

    print(f"Hyperparameters: Embedding Dim={emb_dim}, Hidden Dim={hid_dim}, Learning Rate={lr}, Batch Size={batch_size}, Weight Decay={weight_decay}")


    # Train and validate the model with the current hyperparameters
    validation_loss = train_and_validate(rnn_model, train_iterator, validation_iterator, epochs=8, optimizer=optimizer, loss_fn=loss_fn, device=device)

    # Save the results based on validation performance

    if validation_loss < best_validation_loss:
         best_validation_loss = validation_loss
         torch.save(rnn_model.state_dict(), 'best_model.pth')
         best_hyperparameters = (emb_dim, hid_dim, lr, batch_size, weight_decay)

# Print the best hyperparameters
#print("Best Hyperparameters:")
#print("Embedding Dimension:", best_hyperparameters[0])
#print("Hidden Dimension:", best_hyperparameters[1])
#print("Learning Rate:", best_hyperparameters[2])
#print("Batch Size:", best_hyperparameters[3])
#print("Batch Size:", best_hyperparameters[4])
#print("Best Validation Loss:", best_validation_loss)


Hyperparameters: Embedding Dim=300, Hidden Dim=128, Learning Rate=0.001, Batch Size=64, Weight Decay=0.001
=> Starting training
=> Epoch: 1, Loss: 0.6937156915664673
=> Validation/Test Accuracy: 57.81%
=> Epoch: 2, Loss: 0.6811763644218445
=> Validation/Test Accuracy: 59.50%
=> Epoch: 3, Loss: 0.6757528185844421
=> Validation/Test Accuracy: 59.44%
=> Epoch: 4, Loss: 0.6721706986427307
=> Validation/Test Accuracy: 64.12%
=> Epoch: 5, Loss: 0.6575832962989807
=> Validation/Test Accuracy: 66.88%
=> Epoch: 6, Loss: 0.6511316895484924
=> Validation/Test Accuracy: 65.78%
=> Epoch: 7, Loss: 0.6500775218009949
=> Validation/Test Accuracy: 67.00%
=> Epoch: 8, Loss: 0.643155038356781
=> Validation/Test Accuracy: 66.16%
Hyperparameters: Embedding Dim=200, Hidden Dim=256, Learning Rate=0.01, Batch Size=32, Weight Decay=0.0001
=> Starting training
=> Epoch: 1, Loss: 0.7310294508934021
=> Validation/Test Accuracy: 57.50%
=> Epoch: 2, Loss: 0.6681313514709473
=> Validation/Test Accuracy: 68.03%
=> Ep

In [ ]:
# Define an RNN Model
input_dim = input_dim  # Vocabulary size for input
embedding_dim = 200  #  Size of the dense word vectors created by the embedding layer. A value of 100 is a common choice for word embeddings.
hidden_dim = 256  # Size of the hidden states in the LSTM
output_dim = 1  # Binary classification, so output_dim is 1

RNN_Model = RNN(input_dim=input_dim, embedding_dim=embedding_dim, hidden_dim=hidden_dim, output_dim=output_dim)

# Define loss function and optimizer
lr=.001
loss_fn = nn.BCEWithLogitsLoss() #Binary Cross Entropy with logits
optimizer = optim.Adam(RNN_Model.parameters(), lr = lr, weight_decay=1e-5) #weight decay adds l2 regularization

# Train RNN Model
train_and_validate(RNN_Model, train_iterator, validation_iterator, epochs=10, optimizer=optimizer, loss_fn=loss_fn, device=device)


=> Starting training
=> Epoch: 1, Loss: 0.6708572506904602
=> Validation/Test Accuracy: 58.36%
=> Epoch: 2, Loss: 0.6327364444732666
=> Validation/Test Accuracy: 60.35%
=> Epoch: 3, Loss: 0.6391869187355042
=> Validation/Test Accuracy: 71.95%
=> Epoch: 4, Loss: 0.5676453709602356
=> Validation/Test Accuracy: 72.02%
=> Epoch: 5, Loss: 0.49592339992523193
=> Validation/Test Accuracy: 84.82%
=> Epoch: 6, Loss: 0.39803487062454224
=> Validation/Test Accuracy: 87.35%
=> Epoch: 7, Loss: 0.34668025374412537
=> Validation/Test Accuracy: 87.62%
=> Epoch: 8, Loss: 0.30968061089515686
=> Validation/Test Accuracy: 89.05%
=> Epoch: 9, Loss: 0.27799373865127563
=> Validation/Test Accuracy: 89.51%
=> Epoch: 10, Loss: 0.257412850856781
=> Validation/Test Accuracy: 89.58%


0.27645012523446766

# Test RNN

In [ ]:
# Assuming your model has been trained
evaluate(RNN_Model, test_iterator, loss_fn, device)

=> Validation/Test Accuracy: 89.93%


0.27022369158796117

# Compare RNN Results to 'Baseline Model'

In [ ]:
# Step 1: Determine probabilities in the training dataset
positive_prob = (train_df['sentiment'] == 1).mean()
negative_prob = 1 - positive_prob

# Step 2: Randomly assign labels based on probabilities in the test dataset
np.random.seed(42)  # For reproducibility

# Generate random labels for each row in the test dataset based on probabilities
test_df['predicted_sentiment'] = np.random.choice([0, 1], size=len(test_df), p=[negative_prob, positive_prob])

# Display the baseline predictions
print(test_df[['sentiment', 'predicted_sentiment']].head())

# Calculate loss for the baseline model
baseline_loss = nn.BCEWithLogitsLoss()(torch.tensor(test_df['predicted_sentiment'].values, dtype=torch.float32),
                                       torch.tensor(test_df['sentiment'].values, dtype=torch.float32))

# Calculate accuracy for the baseline model
baseline_correct_predictions = (test_df['predicted_sentiment'] == test_df['sentiment']).sum().item()
baseline_total_samples = len(test_df)
baseline_accuracy = baseline_correct_predictions / baseline_total_samples

print(f'Baseline Model Loss: {baseline_loss:.4f}, Accuracy: {baseline_accuracy * 100:.2f}%')


       sentiment  predicted_sentiment
33553          1                    0
9427           1                    1
199            0                    1
12447          1                    1
39489          0                    0
Baseline Model Loss: 0.7509, Accuracy: 49.83%


# Test RNN on Shifted Dataset (Dataset Shift 1)

In [ ]:
#Dataset Shift 1

# Function to remove words starting with "B", "W", and "S" from a text
def remove_words_starting_with_B_W_S(text):
    words = text.split()

    # Counters for removed words
    removed_B_count = 0
    removed_W_count = 0
    removed_S_count = 0

    filtered_words = []
    for word in words:
        lower_word = word.lower()
        if lower_word.startswith('b'):
            removed_B_count += 1
        elif lower_word.startswith('w'):
            removed_W_count += 1
        elif lower_word.startswith('s'):
            removed_S_count += 1
        else:
            filtered_words.append(word)

    modified_text = ' '.join(filtered_words)
    return modified_text, removed_B_count, removed_W_count, removed_S_count

# Create shifted df
df_dataset_shift = df.copy()

# Split the subset into training, validation, and testing sets
train_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
train_df, validation_df = train_test_split(train_df, test_size=0.2, random_state=42)


# Apply the function to the 'review' column
df_dataset_shift['review'], removed_B_count, removed_W_count, removed_S_count = zip(*df_dataset_shift['review'].apply(remove_words_starting_with_B_W_S))

# Print the modified DataFrame
print(df_dataset_shift.head())

# Calculate the total counts of removed words
total_removed_B_count = sum(removed_B_count)
total_removed_W_count = sum(removed_W_count)
total_removed_S_count = sum(removed_S_count)

# Print the counts of removed words
print(f"Number of words starting with 'B' removed: {total_removed_B_count}")
print(f"Number of words starting with 'W' removed: {total_removed_W_count}")
print(f"Number of words starting with 'S' removed: {total_removed_S_count}")



                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A little production. <br /><br />The filming t...  positive
2  I thought this a to time on a too hot in the a...  positive
3  there's a family a little (Jake) thinks there'...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
Number of words starting with 'B' removed: 531126
Number of words starting with 'W' removed: 689854
Number of words starting with 'S' removed: 786510


In [ ]:
# Function to remove letters from words in a text
def remove_letters_from_words(text, letters_to_remove=['b', 'w', 's', 'e']):
    words = text.split()

    # Counters for removed letters
    removed_count = {letter: 0 for letter in letters_to_remove}

    filtered_words = []
    for word in words:
        lower_word = word.lower()
        modified_word = ''.join(char for char in lower_word if char not in letters_to_remove)

        for letter in letters_to_remove:
            # Update removed count based on occurrences in the word
            removed_count[letter] += lower_word.count(letter)

        filtered_words.append(modified_word)

    modified_text = ' '.join(filtered_words)
    return modified_text, removed_count

# Create a copy of the original DataFrame
df_dataset_shift = df.copy()

# Initialize counters for removed letters
letters_to_remove=['b', 'w', 's', 'e']
total_removed_count = {letter: 0 for letter in letters_to_remove}

# Apply the function to the 'review' column
df_dataset_shift['review'], removed_count = zip(*df_dataset_shift['review'].apply(remove_letters_from_words))

# Update total removed count
for letter in letters_to_remove:
    total_removed_count[letter] = sum(count[letter] for count in removed_count)

# Print the modified DataFrame
print(df_dataset_shift.head())

# Print the counts of removed letters
print("\nCounts of Removed Letters:")
for letter, count in total_removed_count.items():
    print(f"Number of letters '{letter}' removed: {count}")



                                              review sentiment
0  on of th othr rvir ha mntiond that aftr atchin...  positive
1  a ondrful littl production. <r /><r />th filmi...  positive
2  i thought thi a a ondrful ay to pnd tim on a t...  positive
3  aically thr' a family hr a littl oy (jak) thin...  negative
4  pttr matti' "lov in th tim of mony" i a viuall...  positive

Counts of Removed Letters:
Number of letters 'b' removed: 1018636
Number of letters 'w' removed: 1006607
Number of letters 's' removed: 3461798
Number of letters 'e' removed: 6098660


In [ ]:
#'review' is the feature column and 'sentiment' is the label column
X = df_dataset_shift['review']
y = df_dataset_shift['sentiment']

# Convert string labels to binary format
y_binary = y.apply(lambda label: 1 if label == 'positive' else 0)

# Create a new DataFrame with the processed labels
df_processed = pd.DataFrame({'review': X, 'sentiment': y_binary})

# Create a subset for prototyping
subset_size = 30000
#df_processed = df_processed.head(subset_size)
df_processed = df_processed


In [ ]:
# Split the subset into training, validation, and testing sets
train_df, test_df = train_test_split(df_processed, test_size=0.2, random_state=42)
train_df, validation_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Convert DataFrames to CSV files
train_df.to_csv('/content/train.csv', index=False)
validation_df.to_csv('/content/validation.csv', index=False)
test_df.to_csv('/content/test.csv', index=False)

# Create TabularDataset
train_dataset, validation_dataset, test_dataset = TabularDataset.splits(
    path='/content/',
    train='train.csv',
    validation = 'validation.csv',
    test='test.csv',
    format='csv',
    fields=[('review', TEXT), ('sentiment', LABEL)],
    skip_header=True
)


#Dataload the new shifted test dataset
validation_iterator, test_iterator = data.BucketIterator.splits(
      (validation_dataset, test_dataset),
      batch_size=batch,
      sort_within_batch=True,
      sort_key=lambda x: len(x.review),
      device=device
  )


# Assuming model has been trained with non-shifted data - test on shifted dataset
evaluate(RNN_Model, test_iterator, loss_fn, device)

=> Evaluation Loss: 0.6381, Accuracy: 66.09%


0.6381319303801105

GRAVEYARD

In [ ]:
test_iterator_length = len(test_iterator)
print(f"Length of test_iterator: {test_iterator_length}")


In [ ]:
for batch in train_iterator:
    # Process your batch here
    text, text_lengths = batch.review
    labels = batch.sentiment

In [ ]:
#Tokenize the reviews using nltk
df['tokenized_reviews'] = df.apply(lambda row: nltk.word_tokenize(row['review']), axis = 1)

#find the length
df['review_length'] = df.apply(lambda row: len(row['tokenized_reviews']), axis = 1)

df.head()

In [ ]:
# Create a PyTorch dataset with TabularDataset
fields = [('review', TEXT), ('sentiment', LABEL)]
train_data = TabularDataset(path=csv_path, format='csv', fields=fields)

# Build vocabulary
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d", unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)


In [ ]:
# Create iterators
BATCH_SIZE = 64
train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, train_data),
    batch_size=BATCH_SIZE,
    sort_within_batch=False,  # Set to False
    sort_key=lambda x: len(x.review),  # Sort by the length of the review
    device=device
)

# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=1, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        return self.fc(hidden)

# Initialize the model
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1  # Binary classification
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

# Copy pre-trained word embeddings to the embedding layer
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

# Move the model and criterion to the GPU if available
model = model.to(device)
criterion = criterion.to(device)


In [ ]:
# Training loop
N_EPOCHS = 5
for epoch in range(N_EPOCHS):
    train_loss = 0
    train_corrects = 0
    model.train()

    for batch in train_iterator:
        optimizer.zero_grad()
        text, text_lengths = batch.review
        predictions = model(text, text_lengths).squeeze(1)
        loss = criterion(predictions, batch.sentiment.float())
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Convert predictions to binary (0 or 1) and compare with ground truth
        binary_predictions = (predictions >= 0.5).long()
        train_corrects += (binary_predictions == batch.sentiment).sum().item()

    # Calculate training accuracy
    train_accuracy = train_corrects / len(train_iterator.dataset)

    print(f'Epoch {epoch + 1}/{N_EPOCHS}, '
          f'Training Loss: {train_loss / len(train_iterator):.4f}, '
          f'Training Accuracy: {train_accuracy * 100:.2f}%')

